# Anomaly Detection - Parzen Window Density Estimation 

1. 확률 밀도 함수에 대한 직접적인 식을 구한다. 
> $p(x) = \frac{1}{N} \sum_{i=1}^N K(\frac{x^i - x}{h} $ 

2. 다양한 Kernel 함수에 대해 적용할 수 있도록 Kernel_type을 입력 값으로 받으며, 각 Kernel 함수를 구현한다. 
> ex)- $K(u) = \frac{1}{2}1(if |u| <= 1 )$

3. 적절한 h의 값(Smoothing parameter)을 찾기 위해 E-M alghrithm 과정을 진행한다. 

> E : parameter들을 고정하여 객체 확률 탐색. (h -> p(x)) 

> $p(x) = \frac{1}{N} \sum_{i=1}^N K(\frac{x^i - x}{h} $ 


> M : 객체 확률을 토대로 Parameter 탐색 (p(x) - > h) 

> k-nearest neighbor density estimation 계산하기. N * p(x) = k 

> def k_nearest 함수 설정 : k개의 점을 포함하도록 하는 h의 값 산출.  n x 1 벡터 형태로 나오겠지만 임의로 평균 값을 산출 

4. 적절한 h의 값을 토대로 최종 p(x) 값 산출. $x_new$ 값을 대입 시 확률 값이 나오도록 함수 설정 


**구현해야하는 것**
- $p(x) = \frac{1}{N} \sum_{i=1}^N K(\frac{x^i - x}{h}) $ 
- def k_nearest
- 다양한 K(u)
> u가 2차원 이상의 차원일 경우 어떻게 계산할 것인가? 

- 적정한 h 찾았을 시 최종 p(x) 식을 정하는 함수 

**필요한 것**
- X
- kernel_type 
- x_new 
- 임의의 h값. 최초에는 1로 설정 


**함수의 형태**
- def __init__(self, X, kernel_type) : 

- def kernel(self, kernel_type, u) : => 커널 값 반환 
> u = $\frac{x_i - x}{h}$ 

- def p(self, x, kernel_type, h) : => p(x) 값 반환 

- def k_nearest(self, p) : => h 값 반환 

- def find_h(self, epsilon= 1e-10) : h의 값의 차이가 epsilon값보다 작아질때까지 E-M 알고리즘 반복 

- def check_abnormal(self, x_new) : 입력한 x_new 값에 대한 객체 확률 값 반환 




In [1]:
import numpy as np
import pandas as pd
import random as rand

from sklearn.datasets import load_iris
X = load_iris()['data']

import matplotlib.pyplot as plt
import scipy as sc
from scipy.stats import norm
from sys import maxsize

In [51]:
class parzen_window() : 
    def __init__(self, X, kernel_type) : 
        self.X = X 
        self.n = np.shape(X)[0]
        self.m = np.shape(X)[1]
        
        self.kernel_type = kernel_type 
        self.h = 1 #임의로 설정한 값. 
        self.p = np.array([self.cal_p(kernel_type, self.X[i], self.h) for i in range(self.n)]) 
        
    def kernel(self, kernel_type, x, h) : 
        
        u_matrix = (np.array(self.X) - np.array(x)) /h # n x m 형태의 행렬 
        u = [ np.sqrt((u_matrix[i] ** 2).sum()) for i in range(self.n)] # 각 u_matrix의 벡터들의 길이를 모음
        if kernel_type == "Triangular" : 
            k_value = [1-u[i] if u[i] <= 1 else 0 for i in range(self.n)]
        
        elif kernel_type == "Epanechnikov" : 
            k_value = [3*(1-u[i]**2)/4 if u[i] <= 1 else 0 for i in range(self.n)]

        elif kernel_type == "Quartic" : 
            k_value = [15*(1-u[i]**2)/16 if u[i] <= 1 else 0 for i in range(self.n)]
            
        elif kernel_type == "gaussian" : 
            k_value = [np.exp(-u[i]**2/2)/(np.sqrt(2*np.pi)) if u[i] <= 1 else 0 for i in range(self.n)]
        
        else : 
            k_value = [ 1/2 if u[i] <= 1 else 0 for i in range(self.n)]
        
        return k_value 
    
    
    def cal_p(self,  kernel_type, x, h) : # Expectation 역할 
        return np.sum(self.kernel(kernel_type, x,h)) / self.n
    
    def k_nearest(self, x, p) : # Maximization 역할 
        k = int(self.n * p)
        dist_matrix= (np.array(self.X) - np.array(x))
        dist = np.array([ np.sqrt(np.sum(dist_matrix[i] ** 2)) for i in range(self.n)])
        index = np.argsort(dist)
        h = 2*dist[index][k] # h/2 는 반지름으로, x_i - x의 길이가 k 번째인 경우를 포함하도록 설정한다. 
        return h 
    
    def find_h(self, epsilon=1e-10) : 
        pre = self.h 
        
        self.h = np.mean([self.k_nearest(self.X[i], self.p[i]) for i in range(self.n)])
        
        while pre - self.h > epsilon : 
            pre = self.h
            self.p = np.array([self.cal_p(self.kernel_type, self.X[i], self.h) for i in range(self.n)])
            self.h = np.mean([self.k_nearest(self.X[i], self.p[i]) for i in range(self.n)])
       
        return self.h
    
    def check_abnormal(self, x_new, epsilon = 0.01) : 
        p = self.cal_p(self.kernel_type, x_new, self.h)
        
        if p < epsilon : 
            return print(x_new, "is abnormal data. p is ", p)
        
        else : 
            return print(x_new, "is normal data. p is", p)
        
        
        

In [52]:
test = parzen_window(X, "")
test.find_h()
test.check_abnormal( [6.2, 2.9, 4.3, 1.3])



[6.2, 2.9, 4.3, 1.3] is normal data. p is 0.20333333333333334
